In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import operator
from tqdm import tqdm

# import graph data into a pandas dataframe
df_dblp = pd.read_csv(r"D:\Facultate\Year 2\Semester 1\Social Network Analysis for CS\main project\data\com-dblp.ungraph.txt", 
                 sep='\t') # Dont use hardcoded paths, what we did is create a zip of the data, but it is too big so you can just add it to drive or something and then unzip it in the project when you need to run the code.

# rename columns
df_dblp.rename(columns={'# FromNodeId': 'source', 'ToNodeId': 'target'}, inplace=True) # not all datasets have this format, so will need to change

# create networkx graph
g_dblp = nx.from_pandas_edgelist(df_dblp, "source", "target", create_using = nx.Graph)

# I would highly recommend using a more object oriented approach for the code, because now you have to take naming into account and you've got two files and a main file with the same code.
# Also don't forget to clean up unused packages etc.

In [3]:
def get_graph_statistics(g):
    n_nodes = g.number_of_nodes()
    n_edges = len(list(g.edges())) # Isn't there also a number_of_edges() method?
    density = nx.density(g)
    avg_clustering = nx.average_clustering(g)

    
    stats = pd.DataFrame()
    stats["name"] = ["n_nodes", "n_edges","density","avg_clustering"]
    stats["values"] = [n_nodes, n_edges, density, avg_clustering]
    
    return stats
    
stats = get_graph_statistics(g_dblp)

## Strategy 1 - Iteratively removing covered nodes

In [4]:
def strategy1(graph, alpha = 0.00005): # Maybe come up with a name for your strategies so they are easier to identify
    """
    This function selects landmarks and computes the distances from each landmark to every reachable node in the graph.
    
    :param graph (nx.Graph): the networkx graph created from the edgelist
    :param alpha (float): landmark scaling factor. This variable controls the number of landmarks selected at every iteration.
                          The number of landmarks selected is computed as following: N = int(num_remaining_nodes * alpha)

    :returns:
      - distance_mapping (pandas.core.frame.DataFrame): a pandas DataFrame having as index column the list of all nodes in
                                                        the graph. Each column other than the index column is denoted by a
                                                        landmark and contains the distance from the landmark to every other node.
                                                        If a node is not reachable by a landmark, the value in the cell will 
                                                        be NaN.
                                
    """
    
    # get the nodelist
    nodelist = np.array(list(graph.nodes())) # Currently not used
    
    # initialize set of landmarks
    landmark_nodes = []

    # rank the nodes based on degree centrality
    ranking = dict(sorted(nx.degree_centrality(graph).items(), key=operator.itemgetter(1),reverse=True))
    
    # create a list of nodes ordered by their degree centrality
    ranking_list = list(ranking)
    
    current_n_landmarks = 0
    
    # this variable will contain the distances between landmarks and all other nodes
    # if a node is not reachable, the distance will be set to NaN
    distance_mapping = pd.DataFrame()
    distance_mapping["vertices"] = list(graph.nodes())
    distance_mapping = distance_mapping.set_index("vertices")
    
    while len(ranking_list) > 0:
        u_list = ranking_list[:int(len(ranking_list)*alpha)]
        
        for u in tqdm(u_list):
            # add u to the list of landmark nodes
            landmark_nodes.append(u)
            
            # get the distance from "u" to all other nodes
            shortest_path_lengths = nx.single_source_shortest_path_length(graph, u)
            sp_array = np.array(list(shortest_path_lengths.items()))
            
            # get an array of reached nodes
            reached_nodes = list(shortest_path_lengths.keys())
            recorded_distances  = list(shortest_path_lengths.values())
            
            df_u = pd.DataFrame()
            df_u["vertices"] = reached_nodes
            df_u[str(u)] = recorded_distances
            
            distance_mapping = distance_mapping.join(df_u.set_index("vertices"))
            
            # compute the average distance
            average_distance = sp_array[:, 1].mean()

            # get the nodes within average distance
            nodes_in_range = list(sp_array[np.where(sp_array[:,1] <= average_distance)[0], :][:,0])

            # remove the nodes that are within average distance
            updated_ranking_list = set(ranking_list) - set(nodes_in_range) # Maybe don't convert to the ranking list and just use the dict so you don't have to convert back
            ranking_list = list(updated_ranking_list)
                

        new_n_landmarks = len(landmark_nodes)
        
        if new_n_landmarks == current_n_landmarks:
            break
        else:
            current_n_landmarks = new_n_landmarks

    return distance_mapping

In [5]:
distance_mapping = strategy1(g_dblp, alpha = 0.00008)

100%|██████████| 1/1 [00:01<00:00,  1.21s/it]
0it [00:00, ?it/s]


## Strategy 2 -  Community partitioning

In [6]:
from networkx.algorithms.community import louvain_communities, asyn_fluidc, girvan_newman
import matplotlib.pyplot as plt
from operator import itemgetter
import math

In [7]:
def strategy2(graph, k = 50, seed=123):
    """
    This function implements strategy 2 using Fluid Community detection algorithm.
    
    :param graph (nx.Graph): the networkx graph created from the edgelist.
    :param k (int): indicates the number of communities that need to be found.
    :param seed (int): the random seed used in the community detection algorithm.
    
    :returns:
      - distance_mapping (pandas.core.frame.DataFrame): a pandas DataFrame having as index column the list of all nodes in
                                                        the graph. Each column other than the index column is denoted by a
                                                        landmark and contains the distance from the landmark to every other node.
                                                        If a node is not reachable by a landmark, the value in the cell will 
                                                        be infinity.
    """
    
    # search for communities
    
    ##### Using Louvain Communities #####
#     landmark_communities = louvain_communities(graph, resolution=0.6, seed=123) 

    ##### Using fluid communities #####
    landmark_communities  = list(asyn_fluidc(graph, k = k, seed=seed))
    
    # obtain the size of each community
    community_sizes = np.array([len(comm) for comm in landmark_communities])

    num_landmarks = [math.ceil(100*l/sum(community_sizes)) for l in community_sizes]

    # obtain the degree centrality of all nodes in the graph
    degr_centrality_ranking = dict(sorted(nx.degree_centrality(graph).items(), key=operator.itemgetter(1),reverse=True))
    
    degr_centrality_of_communities = {}
    
    distance_mapping2 = pd.DataFrame()
    distance_mapping2["vertices"] = list(graph.nodes())
    distance_mapping2 = distance_mapping2.set_index("vertices") # Move this to the top of the for loop it is actually used for
    
    all_landmarks = []
        
    for i in range(len(landmark_communities)):
        d = {key: degr_centrality_ranking.get(key) for key in landmark_communities[i]}

        num_l = num_landmarks[i]

        # rank the nodes in the community based on degree centrality
        d = dict(sorted(d.items(), key=operator.itemgetter(1),reverse=True)[:num_l]) # Better name for d, even though it is the name in the paper
        
        all_landmarks += list(d.keys())

        degr_centrality_of_communities[i] = d 
    
    for k in degr_centrality_of_communities.keys():
        landmarks = list(degr_centrality_of_communities[k].keys())

        for u in landmarks:
            shortest_path_lengths = nx.single_source_shortest_path_length(graph, u)

            # get an array of reached nodes
            reached_nodes = list(shortest_path_lengths.keys())
            recorded_distances  = list(shortest_path_lengths.values())

            df_u = pd.DataFrame()
            df_u["vertices"] = reached_nodes
            df_u[str(u)] = recorded_distances # Doet is need to be a string?
                               
            df_u = df_u[df_u["vertices"].isin(list(landmark_communities[k]) + all_landmarks)]

            distance_mapping2 = distance_mapping2.join(df_u.set_index("vertices"))
    
    # replace NaN values to infinity
    distance_mapping2.fillna(np.inf, inplace = True) # In strategy 1 you use np.nan and here you use np.inf, why?
    
    return distance_mapping2, all_landmarks

In [ ]:
# %%timeit
distance_mapping2, all_landmarks = strategy2(g_dblp, k = 50, seed=123)

# get the dataframe containing the distances from landmark to landmark
landmark_to_landmark = distance_mapping2.loc[all_landmarks, :]